Topogenesis by Pirouz,Shervin 

env 불러오기

In [24]:
import os
import topogenesis as tg
import trimesh as tm
import numpy as np
import pyvista as pv
import panel as pn
import pandas as pd 

from ladybug.sunpath import Sunpath #ladybug sunpath 호출
from IPython.display import display, Image

In [25]:
# tri_to_pv 함수 정의: 삼각형 메시(tri_mesh)를 PyVista 형식으로 변환합니다.
def tri_to_pv(tri_mesh):
    # tri_mesh.faces 배열에 각 face의 시작 부분에 3을 추가하여 새로운 배열을 만듭니다.
    # 'constant' 모드로 패딩을 추가하여 각 face의 시작에 3을 삽입합니다.
    faces = np.pad(tri_mesh.faces, ((0, 0), (1, 0)), 'constant', constant_values=3)
    # PyVista의 PolyData 객체를 생성하여, tri_mesh의 정점(vertices)과 패딩된 faces 배열을 사용합니다.
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    
    # 변환된 PyVista 메쉬 객체를 반환합니다.
    return pv_mesh

### 복셀화 
imput OBJ를 topogenesis가 읽을 수 있도록 voxelization

In [26]:
vs = 1000 # voxel size 값을 설정합니다. 각 복셀의 크기를 결정합니다.
unit = [vs, vs, 3000]# 복셀 그리드의 각 방향 크기를 정의합니다. (세로 방향 크기 custom 지정)

mesh = tm.load(os.path.relpath('obj/Building01.obj'))# 메쉬 파일을 불러옵니다. 'Building.obj' 파일을 상대 경로로 불러옵니다.
base_lattice = tg.lattice(mesh.bounds, unit=unit, default_value=1, dtype=int)# 복셀 그리드를 초기화합니다. 메쉬 경계와 unit 크기를 사용합니다.
interior_condition = mesh.contains(base_lattice.centroids) # 복셀 중심점을 기준으로 내부 조건을 확인합니다.
interior_array = interior_condition.reshape(base_lattice.shape)# 내부 조건 배열을 3D 복셀 그리드 형태로 재구성합니다.
interior_lattice = tg.to_lattice(interior_array, base_lattice.minbound, base_lattice.unit)# 내부 배열을 복셀 그리드로 변환합니다.

Save as original_lattice_interior_lattice.csv

In [27]:
csv_path = os.path.relpath('original_lattice/interior_lattice.csv')# CSV 파일 경로를 설정합니다.
directory = os.path.dirname(csv_path)# CSV 저장을 위한 디렉토리가 존재하는지 확인하고, 없으면 생성합니다.
if not os.path.exists(directory):
    os.makedirs(directory)

interior_lattice.to_csv(csv_path)# 내부 복셀 그리드를 CSV 파일로 저장합니다.

voxelization 시각화 & save as png

In [28]:
# # PyVista 플로터를 초기화합니다. Jupyter 노트북에서 실행할 수 있도록 설정합니다.
# p = pv.Plotter(notebook=True)
# # 메쉬를 시각화에 추가합니다. tri_to_pv 함수를 사용하여 변환된 메쉬를 사용합니다.
# p.add_mesh(tri_to_pv(mesh), color='#54b36a', opacity=0.3)
# # 내부 복셀 그리드를 시각화에 추가합니다.
# interior_lattice.fast_vis(p)

# # 스크린샷 파일 경로를 설정합니다.
# png_path = os.path.relpath('screenshots/mesh.png')
# # 스크린샷 저장을 위한 디렉토리가 존재하는지 확인하고, 없으면 생성합니다.
# directory = os.path.dirname(png_path)
# if not os.path.exists(directory):
#     os.makedirs(directory)

# # 시각화 플로팅
# p.show(jupyter_backend="html", return_viewer=True)

In [29]:
# PyVista 플로터를 초기화합니다.
p = pv.Plotter(off_screen=True)  # 오프 스크린 모드로 설정하여 시각화하지 않고 이미지 저장 가능

# 메쉬를 시각화에 추가합니다. tri_to_pv 함수를 사용하여 변환된 메쉬를 사용합니다.
p.add_mesh(tri_to_pv(mesh), color='#54b36a', opacity=0.3)
# 내부 복셀 그리드를 시각화에 추가합니다.
interior_lattice.fast_vis(p)
# 스크린샷 파일 경로를 설정합니다.
png_path = os.path.relpath('screenshots/voxelization.png')

# 스크린샷 저장을 위한 디렉토리가 존재하는지 확인하고, 없으면 생성합니다.
directory = os.path.dirname(png_path)
if not os.path.exists(directory):
    os.makedirs(directory)

# 스크린샷 저장
p.screenshot(png_path)
#p.show()
print(f"Screenshot saved to {png_path}")

Screenshot saved to screenshots\voxelization.png


Custom 요소 import

In [30]:
context_mesh = tm.load(os.path.relpath('obj\context.obj'))  # 주변환경 obj 메시 불러오기
street_pc = tg.cloud_from_csv("pts\pts_str(11).CSV")        # street point as csv (포인트를 csv로 저장) 
entrance_pc = tg.cloud_from_csv("pts\ent_pts01.CSV")       # entrance point as csv (라이노 포인트를 csv로 저장)
envelope_lattice = tg.lattice_from_csv(csv_path)

(Euclidean Distance)
유클리드 거리는 두 점 사이의 직선 거리를 나타내며, 이를 계산하는 공식은 다음과 같습니다:

![Euclidean Distance](image/스크린샷%202024-05-28%20143034.png) <!-- 이미지 경로를 여기에 넣으세요 -->

1. 두 점의 각 좌표 성분의 차이를 계산합니다.
2. 차이의 제곱을 구합니다.
3. 제곱값들을 더합니다.
4. 더한 값을 제곱근으로 계산합니다.

점들 사이의 거리 계산 (배열)

In [31]:
# 모든 복셀의 중심 추출
env_cens = envelope_lattice.centroids_threshold(-1)

# 거리 행렬 초기화
dist_ms = []
# 각 복셀에 대해 ...
for voxel_cen in env_cens:
    # 거리 벡터 초기화 (각 복셀에 대해)
    dist_v = []
    # 각 street 점에 대해 ...
    for street_point in street_pc:
        # 차이 벡터를 계산
        diff = voxel_cen - street_point
        # 성분을 제곱
        diff_p2 = diff**2
        # 성분을 합산
        diff_p2s = diff_p2.sum()
        # 제곱근을 계산하여 거리 계산
        dist = diff_p2s**0.5
        # 거리 벡터에 거리 추가
        dist_v.append(dist)
    # 거리 벡터를 거리 행렬에 추가
    dist_ms.append(dist_v)
# 거리 행렬의 타입을 리스트에서 배열로 변경
dist_ms = np.array(dist_ms)

Top preference 최상층 선호도

In [32]:
shape = envelope_lattice.shape
top_preference = np.zeros(shape)

# x, y 좌표에 대한 모든 z 좌표를 포함하는 리스트 생성
z_coords_list = []

for i in range(shape[0]):
    for j in range(shape[1]):
        z_coords_step = []
        for k in range(shape[2]):
            if envelope_lattice[i, j, k]:
                z_coords_step.append(k)
        if z_coords_step:
            z_coords_list.append([i, j, z_coords_step])

# z 좌표를 스케일링하여 [0.0, 1.0] 범위로 변환
z_coords_scaled = []
for coords in z_coords_list:
    x, y, z_vals = coords
    max_z = shape[2] - 1  # 전체 z 범위의 최대값
    min_z = 0  # 전체 z 범위의 최소값
    range_z = max_z - min_z
    scaled_z_vals = [(z - min_z) / range_z for z in z_vals]  # 전체 높이 기준으로 스케일링
    z_coords_scaled.append([x, y, scaled_z_vals])

# 스케일링된 값을 적절한 격자에 배치
for coords in z_coords_scaled:
    x, y, scaled_z_vals = coords
    original_coords = next(item for item in z_coords_list if item[0] == x and item[1] == y)
    for idx, z in enumerate(original_coords[2]):
        top_preference[x, y, z] = scaled_z_vals[idx]

# 결과를 Lattice 객체로 변환하고 스케일링
top_preference_lattice = tg.to_lattice(top_preference, envelope_lattice)
top_preference_lattice *= envelope_lattice * 1.0
top_preference_lattice /= top_preference_lattice.max()

# CSV 파일로 저장
top_csv_path = os.path.relpath('matrixs/top_pre.csv')
top_preference_lattice.to_csv(top_csv_path)



In [33]:
import pyvista as pv
import numpy as np

p = pv.Plotter(off_screen=True)
l = 250 * (top_preference_lattice - top_preference_lattice.min()) / top_preference_lattice.max()  # 재매핑

grid = pv.ImageData(dimensions=l.shape, origin=l.minbound, spacing=l.unit)  # 공간 참조 생성
grid.point_data["Values"] = l.flatten(order="F")  # 데이터 값을 포인트 데이터에 추가

cloud = pv.PolyData(grid.points)  # 포인트 클라우드 생성
cloud.point_data["Values"] = grid.point_data["Values"]

opacity = np.where(cloud.point_data["Values"] > 0, 1, 0)  # 투명도 배열 생성 (0값은 완전히 투명하게, 나머지는 불투명하게)

p.add_mesh(cloud, scalars="Values", render_points_as_spheres=True,  # 포인트 클라우드를 플로터에 추가
           point_size=10, cmap="coolwarm", opacity=opacity, show_scalar_bar=False)

p.screenshot('screenshots/top_pref.png')
print("Screenshot saved to screenshots/top_pref.png")

Screenshot saved to screenshots/top_pref.png


In [34]:
# # 플로터 초기화
# p = pv.Plotter(off_screen=True)
# l = top_preference_lattice 
# # 재매핑
# l = 250 * (l - l.min()) / l.max()
# # 공간 참조 생성
# grid = pv.ImageData()

# # 격자 차원 설정: 값을 주입하려는 근거로 모양(shape)을 설정
# grid.dimensions = l.shape
# # 데이터 세트의 좌하단 모서리
# grid.origin = l.minbound
# # 각 축을 따라 셀 크기 설정
# grid.spacing = l.unit

# # 데이터 값을 셀 데이터에 추가
# grid.point_data["Distance"] = l.flatten(order="F")  # 격자 평탄화
# # 주변 메쉬 추가
# #p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')

# # 포인트 클라우드의 빠른 시각화
# street_pc.fast_notebook_vis(p)
# # 볼륨 추가
# opacity = np.array([0, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6]) * 1
# p.add_volume(grid, cmap="coolwarm", opacity=opacity, shade=True, show_scalar_bar=False)

# # 시각화 플로팅
# p.show(jupyter_backend="html", return_viewer=True)

bottom preference 지하층 선호도

In [35]:
import numpy as np
import os
import topogenesis as tg

# 기본 설정
shape = envelope_lattice.shape
bottom_preference = np.zeros(shape)

# x, y 좌표에 대한 모든 z 좌표를 포함하는 리스트 생성
z_coords_list = []

for i in range(shape[0]):
    for j in range(shape[1]):
        z_coords_step = []
        for k in range(shape[2]):
            if envelope_lattice[i, j, k]:
                z_coords_step.append(k)
        if z_coords_step:
            z_coords_list.append([i, j, z_coords_step])

# z 좌표를 스케일링하여 [1.0, 0.0] 범위로 변환
z_coords_scaled = []
for coords in z_coords_list:
    x, y, z_vals = coords
    max_z = shape[2] - 1  # 전체 z 범위의 최대값
    min_z = 0  # 전체 z 범위의 최소값
    range_z = max_z - min_z
    scaled_z_vals = [1.0 - (z - min_z) / range_z for z in z_vals]  # 전체 높이 기준으로 스케일링
    z_coords_scaled.append([x, y, scaled_z_vals])

# 스케일링된 값을 적절한 격자에 배치
for coords in z_coords_scaled:
    x, y, scaled_z_vals = coords
    original_coords = next(item for item in z_coords_list if item[0] == x and item[1] == y)
    for idx, z in enumerate(original_coords[2]):
        bottom_preference[x, y, z] = scaled_z_vals[idx]

# 결과를 Lattice 객체로 변환하고 스케일링
bottom_preference_lattice = tg.to_lattice(bottom_preference, envelope_lattice)
bottom_preference_lattice *= envelope_lattice * 1.0
bottom_preference_lattice /= bottom_preference_lattice.max()

# CSV 파일로 저장
bottom_csv_path = os.path.relpath('matrixs/ung_pre.csv')
bottom_preference_lattice.to_csv(bottom_csv_path)

In [36]:
import pyvista as pv
import numpy as np

p = pv.Plotter(off_screen=True)
l = 250 * (bottom_preference_lattice - bottom_preference_lattice.min()) / bottom_preference_lattice.max()  # 재매핑

grid = pv.ImageData(dimensions=l.shape, origin=l.minbound, spacing=l.unit)  # 공간 참조 생성
grid.point_data["Values"] = l.flatten(order="F")  # 데이터 값을 포인트 데이터에 추가

cloud = pv.PolyData(grid.points)  # 포인트 클라우드 생성
cloud.point_data["Values"] = grid.point_data["Values"]

opacity = np.where(cloud.point_data["Values"] > 0, 1, 0)  # 투명도 배열 생성 (0값은 완전히 투명하게, 나머지는 불투명하게)

p.add_mesh(cloud, scalars="Values", render_points_as_spheres=True,  # 포인트 클라우드를 플로터에 추가
           point_size=10, cmap="coolwarm", opacity=opacity, show_scalar_bar=False)

p.screenshot('screenshots/ung_pref.png')
print("Screenshot saved to screenshots/ung_pref.png")

Screenshot saved to screenshots/ung_pref.png


In [37]:
# # 플로터 초기화
# p = pv.Plotter()
# l = bottom_preference_lattice 
# # 재매핑
# l = 250 * (l - l.min()) / l.max()
# # 공간 참조 생성
# grid = pv.ImageData()

# # 격자 차원 설정: 값을 주입하려는 근거로 모양(shape)을 설정
# grid.dimensions = l.shape
# # 데이터 세트의 좌하단 모서리
# grid.origin = l.minbound
# # 각 축을 따라 셀 크기 설정
# grid.spacing = l.unit

# # 데이터 값을 셀 데이터에 추가
# grid.point_data["Distance"] = l.flatten(order="F")  # 격자 평탄화
# # 주변 메쉬 추가
# #p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')

# # 포인트 클라우드의 빠른 시각화
# street_pc.fast_notebook_vis(p)
# # 볼륨 추가
# opacity = np.array([0, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6]) * 1
# p.add_volume(grid, cmap="coolwarm", opacity=opacity, shade=True, show_scalar_bar=False)

# # 시각화 플로팅
# p.show(jupyter_backend="html", return_viewer=True)

배열, 재매핑, csv로 저장

In [38]:
# 각 복셀이 가장 가까운 거리 점까지의 거리를 찾기
min_dist = dist_ms.min(axis=1)
max_distance = min_dist.max()  # 최대 거리 계산

# 최소 거리 목록을 격자로 변환
street_eu_distance_lattice = tg.to_lattice(min_dist.reshape(envelope_lattice.shape), envelope_lattice)

# 외부 복셀의 값을 0으로 설정
envelope_st_dist_lattice = street_eu_distance_lattice * envelope_lattice

# 접근성 점수 계산: 거리가 길수록 높은 값 (반전된 점수 계산)
str_acc_lattice = 1 - (envelope_st_dist_lattice / max_distance)
str_acc_lattice *= envelope_lattice  # 외부 복셀의 값은 0으로 유지

# 거리 접근 격자를 CSV 파일로 저장
csv_path = os.path.relpath('matrixs/str_acc.csv')
str_acc_lattice.to_csv(csv_path)


street accessibility 시각화

In [39]:
import pyvista as pv
import numpy as np

p = pv.Plotter(off_screen=True)
l = str_acc_lattice * envelope_lattice  # 격자 곱셈
l = 250 * (l - l.min()) / l.max()  # 재매핑

grid = pv.ImageData(dimensions=l.shape, origin=l.minbound, spacing=l.unit)  # 공간 참조 생성
grid.point_data["Values"] = l.flatten(order="F")  # 데이터 값을 포인트 데이터에 추가

cloud = pv.PolyData(grid.points)  # 포인트 클라우드 생성
cloud.point_data["Values"] = grid.point_data["Values"]

opacity = np.where(cloud.point_data["Values"] > 0, 1, 0)  # 투명도 배열 생성 (0값은 완전히 투명하게, 나머지는 불투명하게)

p.add_mesh(cloud, scalars="Values", render_points_as_spheres=True,  # 포인트 클라우드를 플로터에 추가
           point_size=10, cmap="coolwarm", opacity=opacity, show_scalar_bar=False)

#street_pc.fast_notebook_vis(p)  # 포인트 클라우드의 빠른 시각화 (필요한 경우)

p.screenshot('screenshots/str_acc.png')
print("Screenshot saved to screenshots/str_acc.png")

Screenshot saved to screenshots/str_acc.png


In [40]:
# 모든 복셀의 중심 추출
env_cens = envelope_lattice.centroids_threshold(-1)

# 거리 행렬 초기화
dist_me = []
# 각 복셀에 대해 ...
for voxel_cen in env_cens:
    # 거리 벡터 초기화 (각 복셀에 대해)
    dist_v = []
    # 각 street 점에 대해 ...
    for entrance_point in entrance_pc:
        # 차이 벡터를 계산
        diff = voxel_cen - entrance_point
        # 성분을 제곱
        diff_p2 = diff**2
        # 성분을 합산
        diff_p2s = diff_p2.sum()
        # 제곱근을 계산하여 거리 계산
        dist = diff_p2s**0.5
        # 거리 벡터에 거리 추가
        dist_v.append(dist)
    # 거리 벡터를 거리 행렬에 추가
    dist_me.append(dist_v)
# 거리 행렬의 타입을 리스트에서 배열로 변경
dist_me = np.array(dist_me)

ent accessibility 입구 근접 선호도

In [41]:
import numpy as np
import topogenesis as tg

# 최소 거리에서 접근성 점수 계산
min_dist = dist_me.min(axis=1)
max_distance = min_dist.max()

# 최소 거리 목록을 격자로 변환
ent_eu_distance_lattice = tg.to_lattice(min_dist.reshape(envelope_lattice.shape), envelope_lattice)

# 외부 복셀의 값을 0으로 설정
envelope_ent_dist_lattice = ent_eu_distance_lattice * envelope_lattice

ent_acc_lattice = (envelope_ent_dist_lattice / max_distance)
ent_acc_lattice *= envelope_lattice  # 외부 복셀의 값은 0으로 유지
ent_acc_lattice = 1 - ent_acc_lattice

# 거리 접근 격자를 CSV 파일로 저장
csv_path = os.path.relpath('matrixs/ent_acc.csv')
ent_acc_lattice.to_csv(csv_path)


In [42]:
import pyvista as pv
import numpy as np

p = pv.Plotter(off_screen=True)
l = ent_acc_lattice * envelope_lattice  # 격자 곱셈
l = 250 * (l - l.min()) / l.max()  # 재매핑

grid = pv.ImageData(dimensions=l.shape, origin=l.minbound, spacing=l.unit)  # 공간 참조 생성
grid.point_data["Values"] = l.flatten(order="F")  # 데이터 값을 포인트 데이터에 추가

cloud = pv.PolyData(grid.points)  # 포인트 클라우드 생성
cloud.point_data["Values"] = grid.point_data["Values"]

opacity = np.where(cloud.point_data["Values"] > 0, 1, 0)  # 투명도 배열 생성 (0값은 완전히 투명하게, 나머지는 불투명하게)

p.add_mesh(cloud, scalars="Values", render_points_as_spheres=True,  # 포인트 클라우드를 플로터에 추가
           point_size=10, cmap="coolwarm", opacity=opacity, show_scalar_bar=False)

entrance_pc.fast_notebook_vis(p)

p.screenshot('screenshots/ent_acc.png')
print("Screenshot saved to screenshots/ent_acc.png")

Screenshot saved to screenshots/ent_acc.png


### Ladybug Sun vector 

In [43]:
sp = Sunpath(longitude=37.541, latitude=37.5665)  # 태양궤도 초기화 (setting location variable) 한국 지정

hoys = []                                         # 일년 동안 각 태양 벡터에 대한 시간 목록 정의
sun_vectors = []
day_multiples = 30

for d in range(65):                              # 일년 중 각 날짜에 대해 ... reduce this for shorter period
    if d % day_multiples == 0:                    # 만약 배수 중 하나라면
        for h in range(11, 17):                   # 오전 10시부터 오후 4시까지의 시간 범위 설정

            hoy = d * 24 + h                      # hoy (올해의 시간) 계산
            sun = sp.calculate_sun_from_hoy(hoy)  # 태양 객체 계산
            sun_vector = sun.sun_vector.to_array()# 태양 벡터 추출 (태양 광선이 향하는 방향)

            if sun_vector[2] < 0.0:               # 태양 벡터의 Z 성분이 양수이면, 태양은 수평선 아래에 있습니다.
                hoys.append(hoy)
                sun_vectors.append(sun_vector)

sun_vectors = np.array(sun_vectors)

Rz = tm.transformations.rotation_matrix(np.radians(36.324), [0, 0, 1]) # 회전 행렬 계산

sun_vectors = tm.transform_points(sun_vectors, Rz)                     # 사이트 회전과 일치하도록 태양 벡터 회전

sun_dirs = -np.array(sun_vectors)  # 태양 벡터에서 태양 방향을 구성하기 위해 numpy 배열로 변환

vox_cens = envelope_lattice.centroids  # 복셀의 중심점 추출
# 모든 복셀에서 모든 태양 방향으로 쏘아야 하므로, 이를 위해 복셀 수만큼 태양 방향을 반복하여 ray_dir(모든 광선 방향의 목록)을 구성
ray_dir = []
ray_src = []
for v_cen in vox_cens:
    for s_dir in sun_dirs:
        ray_dir.append(s_dir)
        ray_src.append(v_cen)
# 방향과 출처 목록을 numpy 배열로 변환
ray_dir = np.array(ray_dir)
ray_src = np.array(ray_src)

print("광선을 쏠 복셀의 수 :", vox_cens.shape)
print("각 복셀 당 광선 수 :", sun_dirs.shape)
print("쏠 광선의 수 :", ray_src.shape)

def compute_intersections_in_chunks(context_mesh, ray_src, ray_dir, chunk_size=1000):
    total_rays = ray_src.shape[0]
    tri_ids = []
    ray_ids = []

    for i in range(0, total_rays, chunk_size):
        start_index = i
        end_index = min(i + chunk_size, total_rays)

        chunk_tri_id, chunk_ray_id = context_mesh.ray.intersects_id(
            ray_origins=ray_src[start_index:end_index],
            ray_directions=ray_dir[start_index:end_index],
            multiple_hits=False
        )

        tri_ids.extend(chunk_tri_id)
        ray_ids.extend(chunk_ray_id + start_index)  # Adjust ray_ids to keep the correct global index

    return np.array(tri_ids), np.array(ray_ids)

    # 컨텍스트 메시와 광선의 교차점 계산
tri_id, ray_id = compute_intersections_in_chunks(context_mesh, ray_src, ray_dir, chunk_size=100)

hits = [0] * len(ray_dir)                    # 광선 방향의 길이만큼 0으로 초기화된 명중 여부 목록 생성

for id in ray_id:                            # 교차점이 있는 광선을 1로 표시
    hits[id] = 1

sun_count = len(sun_dirs)                    # 태양 광선 개수와 복셀 개수 계산
vox_count = len(vox_cens)

vox_sun_acc = []                             # 태양 접근성 값을 저장할 목록 초기화

for v_id in range(vox_count):                # 각 복셀에 대해 반복
    int_count = 0                            # 교차점 개수를 저장할 변수 초기화

    for s_id in range(sun_count):            # 태양 광선에 대해 반복
        r_id = v_id * sun_count + s_id       # 복셀 ID와 태양 ID로부터 광선 ID를 계산

        int_count += hits[r_id]              # 교차점이 있는 광선의 개수를 합산

    sun_access = 1.0 - int_count / sun_count # 교차점이 없는 광선의 비율 계산 (태양을 볼 수 있는 정도)
    vox_sun_acc.append(sun_access)           # 비율을 목록에 추가

hits = np.array(hits)                        # 명중 여부 목록을 넘파이 배열로 변환
vox_sun_acc = np.array(vox_sun_acc)          # 복셀별 태양 접근성 값을 넘파이 배열로 변환

env_all_vox = envelope_lattice.flatten();   # 모든 복셀의 조건을 가져옵니다: 그들이 외피 안에 있는지 여부
all_vox_sun_acc = []                        # 모든 복셀의 태양 접근성 값을 저장할 목록 초기화ㅁ
v_id = 0                                    # v_id: 내부 복셀 목록에서의 복셀 ID

for vox_in in env_all_vox:                  # 모든 복셀에 대해, 각 복셀의 내부 여부를 "vox_in"에 배치
    if vox_in == True:                      # 복셀이 외부에 있었으면 ...
        all_vox_sun_acc.append(vox_sun_acc[v_id])   # 태양 접근성 값을 읽고, 모든 복셀의 태양 접근성 목록에 추가
        v_id += 1                           # 다음 번에 다음 복셀을 읽을 때 복셀 ID에 1을 더합니다.
    else:                                   # 복셀이 내부에 있지 않으면 ...
        all_vox_sun_acc.append(0.0)         # 태양 접근성 값으로 0.0을 추가

sunacc_array = np.array(all_vox_sun_acc)    # 배열로 변환
sunacc_array = sunacc_array.reshape(envelope_lattice.shape)  # 격자 모양으로 변경
sunacc_lattice = tg.to_lattice(sunacc_array, envelope_lattice) # 격자로 변환

print(sunacc_lattice.shape)

# csv_path_sun = os.path.relpath('/content/drive/MyDrive/topotry2/Data/sun_acc.csv') # 태양 access lattice 저장
# sunacc_lattice.to_csv(csv_path_sun)
threshold = 1
filtered_array = np.where(sunacc_array <= threshold, sunacc_array, 0)
filtered_lattice = tg.to_lattice(filtered_array, envelope_lattice)
csv_path_filtered = os.path.relpath('matrixs/sun_acc.csv')
filtered_lattice.to_csv(csv_path_filtered)

l = filtered_lattice

광선을 쏠 복셀의 수 : (28495, 3)
각 복셀 당 광선 수 : (18, 3)
쏠 광선의 수 : (512910, 3)


In [ ]:
import pyvista as pv
import numpy as np

p = pv.Plotter(off_screen=True)
l = 250 * (filtered_lattice - filtered_lattice.min()) / filtered_lattice.max()  # 재매핑

grid = pv.ImageData(dimensions=l.shape, origin=l.minbound, spacing=l.unit)  # 공간 참조 생성
grid.point_data["Values"] = l.flatten(order="F")  # 데이터 값을 포인트 데이터에 추가

cloud = pv.PolyData(grid.points)  # 포인트 클라우드 생성
cloud.point_data["Values"] = grid.point_data["Values"]

opacity = np.where(cloud.point_data["Values"] > 0, 1, 0)  # 투명도 배열 생성 (0값은 완전히 투명하게, 나머지는 불투명하게)

p.add_mesh(cloud, scalars="Values", render_points_as_spheres=True,  # 포인트 클라우드를 플로터에 추가
           point_size=10, cmap="coolwarm", opacity=opacity, show_scalar_bar=False)

p.screenshot('screenshots/sun_acc.png')
print("Screenshot saved to screenshots/sun_acc.png")
#p.show()

Screenshot saved to screenshots/sun_acc.png


In [ ]:
import numpy as np
import os
import topogenesis as tg
from scipy import ndimage

# 기본 설정
shape = envelope_lattice.shape

# 외장재에 가까운 정도를 계산
s_sum = tg.create_stencil("von_neumann", 1, 1)
s_sum.set_index([0, 0, 0], 0)
s_sum.set_index([0, 0, 1], 0)
s_sum.set_index([0, -1, 0], 0)
s_sum.set_index([0, 1, 0], 0)
s_sum.set_index([-1, 0, 0], 0)
s_sum.function = np.sum

neighbour_sum = envelope_lattice.apply_stencil(s_sum, border_condition='pad_outside', padding_value=0)
facade_condition = (neighbour_sum == 0).astype(int)

# 외장재와의 거리를 계산
distance_from_facade = ndimage.distance_transform_edt(1 - facade_condition)
distance_from_facade *= envelope_lattice

# 정규화 및 가중치 계산
max_distance = np.max(distance_from_facade)
normalized_distance = distance_from_facade / max_distance if max_distance != 0 else distance_from_facade
inverse_facade_closeness = normalized_distance

# 격자로 변환 및 CSV 파일로 저장
inverse_facade_closeness_lattice = tg.to_lattice(inverse_facade_closeness, envelope_lattice)
csv_path = os.path.relpath('matrixs/dist_facade.csv')
inverse_facade_closeness_lattice.to_csv(csv_path)

In [ ]:
import pyvista as pv
import numpy as np

p = pv.Plotter(off_screen=True)
l = inverse_facade_closeness_lattice * envelope_lattice  # 격자 곱셈
l = 250 * (l - l.min()) / l.max()  # 재매핑

grid = pv.ImageData(dimensions=l.shape, origin=l.minbound, spacing=l.unit)  # 공간 참조 생성
grid.point_data["Values"] = l.flatten(order="F")  # 데이터 값을 포인트 데이터에 추가

cloud = pv.PolyData(grid.points)  # 포인트 클라우드 생성
cloud.point_data["Values"] = grid.point_data["Values"]

opacity = np.where(cloud.point_data["Values"] > 0, 1, 0)  # 투명도 배열 생성 (0값은 완전히 투명하게, 나머지는 불투명하게)

p.add_mesh(cloud, scalars="Values", render_points_as_spheres=True,  # 포인트 클라우드를 플로터에 추가
           point_size=10, cmap="coolwarm", opacity=opacity, show_scalar_bar=False)

entrance_pc.fast_notebook_vis(p)

p.screenshot('screenshots/dist_facade.png')
print("Screenshot saved to screenshots/dist_facade.png")

Screenshot saved to screenshots/dist_facade.png
